In [2]:
import logging
import hydra
import pandas as pd

from collections import defaultdict
from tqdm import tqdm 


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

with hydra.initialize(version_base=None, config_path="./config"):
    cfg = hydra.compose(config_name="config", overrides={"dataset.generate_csv=True"})
dataset = hydra.utils.instantiate(cfg.dataset)


Processing ECG / EDA / Label:   0%|          | 0/20 [00:00<?, ?it/s]

In [3]:
# DEFAULT VALUE 
choice_key = ['emotion','valence','arousal','gender']




eva_dict = defaultdict(list)
for sample in tqdm(dataset):
    for key_ in choice_key:
        if not sample.get(key_) == None:
            eva_dict[key_].append(sample.get(key_).item())

 11%|█▏        | 1157/10128 [00:02<00:15, 580.11it/s]

Error occurs ->  data/KEMDy19/wav/Session03/Sess03_script04


 25%|██▌       | 2564/10128 [00:04<00:13, 575.35it/s]

Error occurs ->  data/KEMDy19/wav/Session06/Sess06_script02
Error occurs ->  data/KEMDy19/wav/Session06/Sess06_script02


 30%|██▉       | 3031/10128 [00:05<00:12, 574.85it/s]

Error occurs ->  data/KEMDy19/wav/Session07/Sess07_script02
Error occurs ->  data/KEMDy19/wav/Session07/Sess07_script02


100%|██████████| 10128/10128 [00:17<00:00, 574.41it/s]


In [4]:

df = pd.DataFrame(eva_dict)
df

,emotion,valence,arousal,gender
0,1,1.7,4.0,0
1,2,1.5,3.6,1
2,3,1.3,4.3,0
3,3,1.5,3.8,0
4,2,1.6,3.7,1
...,...,...,...,...
10118,6,2.3,2.1,0
10119,6,2.5,2.3,1
10120,6,2.1,1.8,0
10121,5,3.9,3.2,1


## Emotion EDA

In [23]:
# 성별 
print('Male: {:04d}, Female: {:04d}'.format(*df['gender'].value_counts().values))

# Valence min-max 
for key in ['valence','arousal']:
    print(f'{key}: \n\tMin: {min(df[key]):.2f}\n\tMax: {max(df[key]):.2f}')



Male: 5183, Female: 4940
valence: 
	Min: 1.00
	Max: 5.00
arousal: 
	Min: 1.40
	Max: 5.00


In [36]:
emotion_r= { 
1: 'surprise',
 2: 'fear',
 3: 'angry',
 4: 'neutral',
 5: 'happy',
 6: 'sad',
 7: 'disgust'}

In [37]:
df[df['emotion']==0]

,emotion,valence,arousal,gender
43,0,2.2,3.1,0
53,0,1.7,3.6,0
77,0,4.2,3.6,1
92,0,3.1,3.0,1
136,0,2.1,2.7,1
...,...,...,...,...
10021,0,2.0,4.3,0
10022,0,1.7,3.8,1
10052,0,2.3,3.5,1
10073,0,2.2,3.9,0


In [38]:

# 감정 별 Valence Arousal mean, std 
mean_std_dict = defaultdict(dict)
for i in df['emotion'].unique():
    emotion = emotion_r.get(i)
    means = df[df['emotion'] == i][choice_key[1:-1]].mean(axis=0).values
    stds = df[df['emotion'] == i][choice_key[1:-1]].std(axis=0).values

    tmp_dict = {
        'valence_mean': means[0],
        'valence_std' : means[1],
        'arousal_mean' : stds[0],
        'arousal_std': stds[1]
    }
    
    mean_std_dict[emotion] = tmp_dict


In [39]:
mean_std_dict

defaultdict(dict,
            {'surprise': {'valence_mean': 2.6472448948694733,
              'valence_std': 3.6433673479119126,
              'arousal_mean': 0.8856260971230319,
              'arousal_std': 0.5327735883932074},
             'fear': {'valence_mean': 1.7393139764312704,
              'valence_std': 3.6382585710460087,
              'arousal_mean': 0.3645110587205513,
              'arousal_std': 0.47457931560427524},
             'angry': {'valence_mean': 1.8041186092451076,
              'valence_std': 3.7963207056075907,
              'arousal_mean': 0.39539874872737485,
              'arousal_std': 0.586658141782647},
             'neutral': {'valence_mean': 2.968239999548594,
              'valence_std': 2.980586666075389,
              'arousal_mean': 0.5950913754293659,
              'arousal_std': 0.33588868528295596},
             None: {'valence_mean': 2.8068642636729106,
              'valence_std': 3.3146645860999313,
              'arousal_mean': 0.929322499

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt 
from matplotlib import colormaps

from matplotlib.patches import Ellipse


def drawing_ellipse(total_dict):
    fig, ax = plt.subplots(figsize=(5, 5))
    for emotion_key in total_dict.keys():
        x,y, width, hight = ellipse_dict.get(emotion_key)
    tmp = Ellipse((x,y), width, hight,
                  color=color_palette[emotion_key],
                  fill = False,
                  label=emotion_r.get(emotion_key),alpha=0.9) # Ellipse (x,y), width,  hight 
    ax.add_artist(tmp)

ax.set_xlim((1, 5))
ax.set_ylim((1, 5))
ax.legend()
ax.set_xlabel("Valence")
ax.set_ylabel("Arousal")
plt.grid()
plt.show()

In [ ]:
import seaborn as sns 
color_palette = sns.color_palette("Set2")

fig, ax = plt.subplots(figsize=(5, 5))
for emotion_key in ellipse_dict.keys():
    x,y, width, hight = ellipse_dict.get(emotion_key)
    tmp = Ellipse((x,y), width, hight,
                  color=color_palette[emotion_key],
                  fill = False,
                  label=emotion_r.get(emotion_key),alpha=0.9) # Ellipse (x,y), width,  hight 
    ax.add_artist(tmp)

ax.set_xlim((1, 5))
ax.set_ylim((1, 5))
ax.legend()
ax.set_xlabel("Valence")
ax.set_ylabel("Arousal")
plt.grid()
plt.show()

In [ ]:


'''
patches.Ellipse(
       xy = (5, 5), # xy xy coordinates of ellipse centre.
       width = 5,   # width Total length (diameter) of horizontal axis.
       height = 10, # height Total length (diameter) of vertical axis.
       angle = -40, # angle Rotation in degrees anti-clockwise. 0 by default
       edgecolor = 'black',
       linestyle = 'solid', 
       fill = True,
       facecolor = 'yellow',
   ))

'''
import seaborn as sns 
color_palette = sns.color_palette("Set2")

fig, ax = plt.subplots(figsize=(5, 5))
for emotion_key in ellipse_dict.keys():
    x,y, width, hight = ellipse_dict.get(emotion_key)
    tmp = Ellipse((x,y), width, hight,
                  color=color_palette[emotion_key],
                  fill = False,
                  label=emotion_r.get(emotion_key),alpha=0.9) # Ellipse (x,y), width,  hight 
    ax.add_artist(tmp)

ax.set_xlim((1, 5))
ax.set_ylim((1, 5))
ax.legend()
ax.set_xlabel("Valence")
ax.set_ylabel("Arousal")
plt.grid()
plt.show()

# [eneu, ehap, esad, esur, efea, edis, eang], ["Neutral", "Happy", "Sad", "Surprise", "Fear", "Disgust", "Anger"]
# ax.set_xlim((-1, 1))
# ax.set_ylim((-1, 1))
# ax.set_xlabel("Valence")
# ax.set_ylabel("Arousal")
# ax.legend([eneu, ehap, esad, esur, efea, edis, eang], ["Neutral", "Happy", "Sad", "Surprise", "Fear", "Disgust", "Anger"])
# plt.show()

In [ ]:
import seaborn as sns 

# refer : IAPS affective rating
import matplotlib.cm as cm
import matplotlib as mpl

color_palette   = mpl.colormaps['viridis']
# cmap = cm.bwr ## 또는 plt.cm.get_cmap('bwr')
vmax = 6
vmin = 0
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
colormapping = cm.ScalarMappable(norm=norm, cmap=color_palette)


sns_palette = sns.color_palette('Set3')

label_fsize = 20
ticks_fsize = 12
legend_size = 10

plt.figure(figsize=(7,7))
plt.scatter(df['arousal'], df['valence'],
            color=[sns_palette[int(i)] for i in df['emotion']],
            # cmap = colormapping,
            alpha=0.4)


plt.arrow(1.3, 1, 3.2,4, width = 0.05 ,label='Appetitive Motivation', alpha=0.6,color='red')
plt.arrow(2, 3,   3, -2.1,width = 0.05 ,label='Defensive Motivation', alpha=0.6,color='blue')




plt.xlabel('Arousal', fontweight='bold', fontsize=label_fsize)
plt.ylabel('Valence', fontweight='bold', fontsize=label_fsize)
plt.xlim(1,5.3)
plt.ylim(0.7,5.3)
plt.grid(True)
plt.yticks(fontsize=ticks_fsize, fontweight='normal')
plt.xticks(fontsize=ticks_fsize,
            fontweight='normal',
            )
plt.legend(loc='upper left',fontsize=legend_size,)

cb = plt.colorbar(label='Emotions', ax=plt.gca())
# cb.set_ticks([0,1,2,3,4,5,6])
cb.set_ticklabels(['surprise', 'fear', 'angry', 'neutral', 'happy', 'sad', 'disgust'])
# plt.legend(,
            #    loc='upper left',
            #    fontsize=legend_size,
            #    )
plt.show()





In [ ]:
# refer : IAPS affective rating

color_palette = sns.color_palette("Paired")
label_fsize = 20
ticks_fsize = 12
legend_size = 10

plt.figure(figsize=(7,7))


plt.scatter(df['arousal'], df['valence'],
            # color=[f'C{int(i)}' for i in df['emotion']],
            color=[color_palette[int(i)] for i in df['emotion']],
            # edgecolors='black',linewidth=1,
            # label=df['emotion'],
            # label=[f'{emotion_r.get(i)}' for i in df['emotion']],
            alpha=0.4)


# plt.plot([1.3,4.7],[1,5.2],label='Appetitive Motivation', alpha=0.9,color='red')
plt.arrow(1.3, 1, 3.2,4, width = 0.05 ,label='Appetitive Motivation', alpha=0.6,color='red')
# plt.plot([2,5.2],[3,1],label='Appetitive Motivation', alpha=0.7,color='green')
plt.arrow(2, 3,   3, -2.1,width = 0.05 ,label='Defensive Motivation', alpha=0.6,color='green')
# plt.plot(x,z,label='z=3x')
# plt.legend(bbox_to_anchor=(1,1))

# [ 출처: https://seong6496.tistory.com/ ]



plt.xlabel('Arousal', fontweight='bold', fontsize=label_fsize)
plt.ylabel('Valence', fontweight='bold', fontsize=label_fsize)
plt.xlim(1,5.3)
plt.ylim(0.7,5.3)
plt.grid(True)
plt.yticks(fontsize=ticks_fsize, fontweight='normal')
plt.xticks(fontsize=ticks_fsize,
            fontweight='normal',
            )
plt.legend(loc='upper left',fontsize=legend_size,)



from matplotlib.colors import ListedColormap
# cmap = 
# cmap = ListedColormap(sns.color_palette("Paired"))
plt.colorbar(label='Emotions', ax=plt.gca())

# cb = plt.colorbar(label='Emotions', ax=plt.gca())

# cb.set_ticks([0,1,2,3,4,5,6])
# cb.set_ticklabels(, ,['surprise', 'fear', 'angry', 'neutral', 'happy', 'sad', 'disgust'])
# plt.legend(,
            #    loc='upper left',
            #    fontsize=legend_size,
            #    )
plt.show()


In [ ]:
color_palette

In [ ]:
from matplotlib.colors import ListedColormap

cb = fig.colorbar(cmap, label='Emotions')
cb.set_ticks([0,1,2,3,4,5,6])
# cb.set_ticklabels(['surprise', 'fear', 'angry', 'neutral', 'happy', 'sad', 'disgust'])


In [ ]:
# for idx, row in df.iterrows():
#     # print(row[2])
#     plt.scatter(row.valence, row.arousal, color='C{}'.format(int(row.emotion)))

In [ ]:
color_palette